# BART fleet-aware train length and frequency planning

In [2]:
import os
import re
import io
import math
import time
import json
import copy
import random
import zipfile
import datetime as dt
from dataclasses import dataclass
from pathlib import Path
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import networkx as nx

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from IPython.display import display

## Constants

In [ ]:
# Seed
RANDOM_SEED = 222
np.random.seed(RANDOM_SEED)

# Path for managing data loading
PROJECT_DIR = Path(".")
DATA_DIR = PROJECT_DIR / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Data source
OD_BASE_URL = "https://afcweb.bart.gov/ridership/origin-destination/"
OD_FILENAME_TEMPLATE = "date-hour-soo-dest-{year}.csv.gz"
OD_URL_TEMPLATE = OD_BASE_URL + OD_FILENAME_TEMPLATE
OD_FILEPATH = None

# Google Transit Data Source (for round trip times)
GTFS_ZIPPATH = None  # e.g., DATA_DIR / "google_transit.zip"
GTFS_URL = "https://www.bart.gov/dev/schedules/google_transit.zip"

# Date Selection?
TARGET_DATE = "2024-12-31" 
AUTO_DATE_RANGE = ("2024-01-01", "2024-12-31")
WINDOW_WEEKDAYS_ONLY = True

# Time modeling
TIME_MODE = "BLOCK_PEAK_HOUR"  # "BLOCK_PEAK_HOUR" or "HOURLY"?
PERIOD_TO_HOURS = {
    "AM": [6, 7, 8, 9],
    "MID": [10, 11, 12, 13, 14],
    "PM": [15, 16, 17, 18],
    "EVE": [19, 20, 21],
}

# Routing?
# Recommended for "simulate the BART map": allow transfers, but penalize extra transfers
ROUTING_MODE = "SHORTEST_PATH_TRANSFER_PENALTY"
# other options include
# "DIRECT_LINE_ONLY", "SHORTEST_PATH", "SHORTEST_PATH_TRANSFER_PENALTY"
TRANSFER_PENALTY_EDGES = (
    4  # cost of transferring between lines at a station (in "equivalent stops")
)

# Capacity and train lengths
CAP_PER_CAR = 200
K_LIST = list(range(3, 11))  # 3..10 cars

# Fleet Limit
FLEET_CARS_DEFAULT = 1100

# Baseline Train Length
# This is almost just a starting point
BASELINE_LENGTH_BY_LINE_PERIOD = {
    ("YELLOW", "AM"): 9,
    ("YELLOW", "MID"): 8,
    ("YELLOW", "PM"): 9,
    ("YELLOW", "EVE"): 7,
    ("RED", "AM"): 8,
    ("RED", "MID"): 7,
    ("RED", "PM"): 8,
    ("RED", "EVE"): 6,
    ("GREEN", "AM"): 8,
    ("GREEN", "MID"): 7,
    ("GREEN", "PM"): 8,
    ("GREEN", "EVE"): 6,
    ("BLUE", "AM"): 7,
    ("BLUE", "MID"): 6,
    ("BLUE", "PM"): 7,
    ("BLUE", "EVE"): 5,
    ("ORANGE", "AM"): 7,
    ("ORANGE", "MID"): 6,
    ("ORANGE", "PM"): 7,
    ("ORANGE", "EVE"): 5,
}

# Gurobi stuff
PENALTY_MINFREQ = 1e6
MILP_LEXICOGRAPHIC = True
MILP_TIME_LIMIT_SEC = 300  # per phase; lexicographic can use up to ~2x this
ADD_SMOOTHNESS = False
SMOOTHNESS_DELTA_TRAINS_PER_HR = 3

# Simulated annealing heuristic?
SA_ENABLED = True
SA_ITER = 20000
SA_T0 = 1.0
SA_ALPHA = 0.9995
SA_FLEET_PENALTY = 1e7
SA_MINFREQ_PENALTY = 1e7
SA_MAXFREQ_PENALTY = 1e6
SA_PRINT_EVERY = 2000

# Sensitivity analysis
RUN_SENSITIVITY = True
FLEET_GRID = list(range(250, 551, 25))
SENS_TIME_LIMIT_SEC = 90  # per phase; lexicographic can use up to ~2x this
